# Elasticsearch
엘라스틱서치의 기본 원리와 사용법에 대해서 배웁니다.
____

## 1. Elasticsearch의 Python 라이브러리 설치

In [1]:
!pip install elasticsearch

In [2]:
from elasticsearch import Elasticsearch

## 2. Index 세팅
- 한글 데이터를 다루기 때문에, 은전한닢(Mecab) 형태소 분석기를 탑재합니다.

In [3]:
import pprint  
INDEX_NAME = "toy_index"


INDEX_SETTINGS = {
  "settings" : {
    "index":{
      "analysis":{
        "analyzer":{
          "korean":{
            "type":"custom",
            "tokenizer":"nori_tokenizer"
          }
        }
      }
    }
  },
  "mappings": {

      "properties" : {
        "content" : {
          "type" : "text",
          "analyzer": "korean"
        },
        "title" : {
          "type" : "text",
          "analyzer": "korean"
        }
      }

  }
}


## 3. 문서 준비

In [4]:
DOCS = {
    1: {"title": "My Love",
        "content": "사랑해 그 말은 무엇보다 아픈 말 숨죽여서 하는 말 이젠 하기 힘든 말"
        },
    2: {"title": "듣고있나요",
        "content": "끝내 우린 스쳐가나요 기억넘어 서로를 지워야하나요 내게 사랑이 준 깊은 상처는 어떻게 견디며 살아야하는지 매일 아픈 그리움속에 가슴 텅 빈채 살아도 그대를 사랑했던일 그것만은 죽어도 나 후회하지않아요"
        },
    3: {"title": "인연",
        "content": "눈을 떠 바라보아요 그댄 정말 가셨나요 단 한번 보내준 그대 눈빛은 날 사랑했나요 또 다른 사랑이 와도 이젠 쉽게 허락되진 않아 견디기 힘들어 운명같은 우연을"
        },
    4: {"title": "말리 꽃",
        "content": ["얼마나 더 견뎌야 하는지 짙은 어둠을 헤메고 있어","내가 바란 꿈이라는 것은 없는걸까", "더 이상은 견딜수 없는 것"]
        },
    5: {"title": "그런 사람 또 없습니다",
        "content": "천번 번이고 다시 태어난대도 그런 사람 또 없을테죠 슬픈 내삶을 따뜻하게 해준 잠 고마운 사람입니다"
        }
}  

## 4. Elastichsearch 접속

In [5]:

try:
    es.transport.close()
except:
    pass
es = Elasticsearch()



In [6]:
es.info()

{'name': '88f2909b2200',
 'cluster_name': 'elasticsearch',
 'cluster_uuid': 'uTMcRA_ASU6p8zqcAN7Cxg',
 'version': {'number': '7.12.1',
  'build_flavor': 'default',
  'build_type': 'deb',
  'build_hash': '3186837139b9c6b6d23c3200870651f10d3343b7',
  'build_date': '2021-04-20T20:56:39.040728659Z',
  'build_snapshot': False,
  'lucene_version': '8.8.0',
  'minimum_wire_compatibility_version': '6.8.0',
  'minimum_index_compatibility_version': '6.0.0-beta1'},
 'tagline': 'You Know, for Search'}

## 5. 인덱스 생성

In [7]:
if es.indices.exists(INDEX_NAME):
    es.indices.delete(index=INDEX_NAME)
es.indices.create(index=INDEX_NAME, body=INDEX_SETTINGS)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'toy_index'}

## 6. 문서삽입

In [8]:
import time
for doc_id, doc in DOCS.items():
    es.index(index=INDEX_NAME,  id=doc_id, body=doc)
    time.sleep(1)


## 7. 삽입된 문서 확인

In [9]:
doc = es.get(index=INDEX_NAME, id=1)
pprint.pprint(doc)

{'_id': '1',
 '_index': 'toy_index',
 '_primary_term': 1,
 '_seq_no': 0,
 '_source': {'content': '사랑해 그 말은 무엇보다 아픈 말 숨죽여서 하는 말 이젠 하기 힘든 말',
             'title': 'My Love'},
 '_type': '_doc',
 '_version': 1,
 'found': True}


## 8. Term vector 확인

In [10]:
tv = es.termvectors(index=INDEX_NAME, id=2,body={"fields" : ["content","title"]})

In [11]:
pprint.pprint(tv)

{'_id': '2',
 '_index': 'toy_index',
 '_type': '_doc',
 '_version': 1,
 'found': True,
 'term_vectors': {'content': {'field_statistics': {'doc_count': 5,
                                                   'sum_doc_freq': 169,
                                                   'sum_ttf': 216},
                              'terms': {'ᆫ': {'term_freq': 3,
                                              'tokens': [{'end_offset': 36,
                                                          'position': 21,
                                                          'start_offset': 35},
                                                         {'end_offset': 64,
                                                          'position': 35,
                                                          'start_offset': 62},
                                                         {'end_offset': 77,
                                                          'position': 42,
                                    

## 7. 검색

In [12]:
query = "사랑하지만 힘들어 죽겠네"
res = es.indices.analyze(index=INDEX_NAME,
                                 body={
                                       "tokenizer" : "nori_tokenizer",
                                        "text" : query
                                 }
                        )
pprint.pprint(res)

{'tokens': [{'end_offset': 2,
             'position': 0,
             'start_offset': 0,
             'token': '사랑',
             'type': 'word'},
            {'end_offset': 3,
             'position': 1,
             'start_offset': 2,
             'token': '하',
             'type': 'word'},
            {'end_offset': 5,
             'position': 2,
             'start_offset': 3,
             'token': '지만',
             'type': 'word'},
            {'end_offset': 8,
             'position': 3,
             'start_offset': 6,
             'token': '힘들',
             'type': 'word'},
            {'end_offset': 9,
             'position': 4,
             'start_offset': 8,
             'token': '어',
             'type': 'word'},
            {'end_offset': 11,
             'position': 5,
             'start_offset': 10,
             'token': '죽',
             'type': 'word'},
            {'end_offset': 12,
             'position': 6,
             'start_offset': 11,
             'token':

In [13]:
res = es.search(index=INDEX_NAME, q=query, size=10)
pprint.pprint(res)

{'_shards': {'failed': 0, 'skipped': 0, 'successful': 1, 'total': 1},
 'hits': {'hits': [{'_id': '2',
                    '_index': 'toy_index',
                    '_score': 2.5875735,
                    '_source': {'content': '끝내 우린 스쳐가나요 기억넘어 서로를 지워야하나요 내게 사랑이 '
                                           '준 깊은 상처는 어떻게 견디며 살아야하는지 매일 아픈 '
                                           '그리움속에 가슴 텅 빈채 살아도 그대를 사랑했던일 그것만은 '
                                           '죽어도 나 후회하지않아요',
                                'title': '듣고있나요'},
                    '_type': '_doc'},
                   {'_id': '3',
                    '_index': 'toy_index',
                    '_score': 2.3695383,
                    '_source': {'content': '눈을 떠 바라보아요 그댄 정말 가셨나요 단 한번 보내준 그대 '
                                           '눈빛은 날 사랑했나요 또 다른 사랑이 와도 이젠 쉽게 허락되진 '
                                           '않아 견디기 힘들어 운명같은 우연을',
                                'title': '인연'},
                    '_type': '_doc'},


In [14]:
for hit in res['hits']['hits']:
    print("Doc ID: %3r  Score: %5.2f" % (hit['_id'], hit['_score']))


Doc ID: '2'  Score:  2.59
Doc ID: '3'  Score:  2.37
Doc ID: '1'  Score:  1.88
Doc ID: '4'  Score:  0.67
Doc ID: '5'  Score:  0.12
